In [1]:
##### ETL NOTEBOOK FOR 2023 MHSAA TOURNEY SPECIFIC MAP

#### Adapted from ETL for JSON

## Dependencies and Setup
### Dependencies

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import re
import time

## Start timer
start_time = time.time()


In [2]:
## LOAD BLOCK###
#### Load data from kml file exported from Google Earth

file_path = ('data/kml/MHSAA_2023.kml') # file path to kml file


# Read the KML file
with open(file_path) as file:
    xml_data = file.read()

# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')
folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')

# Create a list to store rows to append to the DataFrame
rows = []

# Loop through the folders and extract the data
for folder in list:
    try:
        field_name = folder.find('name').text
        foul = folder.find_all('coordinates')[0].text
        fop = folder.find_all('coordinates')[1].text
        notes = None

        # Check if there is a description tag, if so, use it for notes
        if folder.find('description') is not None:
            notes = folder.find('description').text

        row = {
            'field': field_name,
            'foul': foul,
            'fop': fop,
            'notes': notes
        }

        rows.append(row)

    except Exception as e:
        # Add name of folder to a list of failed folders
        failed.append(folder.find('name').text)
        print(f"Error processing folder: {folder.find('name').text}. Error message: {str(e)}")

# Convert the list of rows to a DataFrame
df = pd.DataFrame(rows, columns=['field', 'foul', 'fop', 'notes'])

# print('Failed to parse:', failed)


In [3]:
df.head()

,field,foul,fop,notes
0,Muskegon Western Michigan Christian HS,"\n\t\t\t\t\t\t\t\t-86.23679559999999,43.160902...","\n\t\t\t\t\t\t\t\t-86.23679559999999,43.160902...",None
1,St Joseph HS,"\n\t\t\t\t\t\t\t\t-86.4923405,42.0912197,0 -86...","\n\t\t\t\t\t\t\t\t-86.4923405,42.0912197,0 -86...",no fence in foul outfield
2,Petoskey High School,"\n\t\t\t\t\t\t\t\t-84.93868740000001,45.367230...","\n\t\t\t\t\t\t\t\t-84.93868740000001,45.367230...",None
3,U of M - college,"\n\t\t\t\t\t\t\t\t-83.7420742,42.2678568,0 -83...","\n\t\t\t\t\t\t\t\t-83.7420742,42.2678568,0 -83...",None
4,Sagniaw Valley State - college,"\n\t\t\t\t\t\t\t\t-83.95339989999999,43.512229...","\n\t\t\t\t\t\t\t\t-83.95339989999999,43.512229...",None


In [4]:
# Clean the new dataframe


# Create a copy of the original DataFrame
df_cleaned = df.copy()

# Remove new line and space characters from coordinates
df_cleaned = df_cleaned.replace(r'\n','', regex=True) 
df_cleaned = df_cleaned.replace(r'\t','', regex=True) 

# Drop any duplicate rows
df_cleaned = df_cleaned.drop_duplicates(subset=['field'], keep='first')

# Drop any rows with empty fields
df_cleaned = df_cleaned[(df_cleaned != 0).all(1)]

In [5]:
df_cleaned.info()
df_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 0 to 130
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   field   127 non-null    object
 1   foul    127 non-null    object
 2   fop     127 non-null    object
 3   notes   4 non-null      object
dtypes: object(4)
memory usage: 5.0+ KB


,field,foul,fop,notes
0,Muskegon Western Michigan Christian HS,"-86.23679559999999,43.1609022,0 -86.236754,43....","-86.23679559999999,43.1609022,0 -86.2356235,43...",None
1,St Joseph HS,"-86.4923405,42.0912197,0 -86.4923378,42.092111...","-86.4923405,42.0912197,0 -86.4911402,42.091221...",no fence in foul outfield
2,Petoskey High School,"-84.93868740000001,45.36723030000001,0 -84.938...","-84.93868740000001,45.36723030000001,0 -84.937...",None
3,U of M - college,"-83.7420742,42.2678568,0 -83.74204469999999,42...","-83.7420742,42.2678568,0 -83.74325570000001,42...",None
4,Sagniaw Valley State - college,"-83.95339989999999,43.5122297,0 -83.9533737,43...","-83.95339989999999,43.5122297,0 -83.9521446,43...",None


In [6]:
##### Clean up polygon data and create a new home_plate column

def parse_coordinates(coord_string):
    coords = coord_string.split()
    parsed_coords = [tuple(map(float, coord.split(',')[:2])) for coord in coords]
    return parsed_coords

# Create a new column for the home_plate location using the first set of coordinates in the 'fop' column
df_cleaned['home_plate'] = df_cleaned['fop'].apply(lambda x: parse_coordinates(x)[0])

# Apply the parse_coordinates function to the 'foul' and 'fop' columns
df_cleaned['foul'] = df_cleaned['foul'].apply(parse_coordinates)
df_cleaned['fop'] = df_cleaned['fop'].apply(parse_coordinates)


In [7]:
############## AREA CALCULATION ##############


import pyproj
from shapely.geometry import Polygon
from shapely.ops import transform


def calculate_area(coords):
    # Create a Polygon object from the coordinates
    polygon = Polygon(coords)

    # Calculate the centroid of the polygon
    centroid = polygon.centroid

    # Create a custom LAEA projection centered on the centroid
    custom_projection = f"+proj=laea +lat_0={centroid.y} +lon_0={centroid.x} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

    # Create a transformer for converting coordinates to the custom LAEA projection
    transformer = pyproj.Transformer.from_crs(
        pyproj.CRS("EPSG:4326"),  # WGS 84 (latitude and longitude)
        pyproj.CRS(custom_projection),  # Custom LAEA projection
        always_xy=True
    )

    # Define a function to transform coordinates using the transformer
    def transform_coordinates(x, y):
        return transformer.transform(x, y)

    # Convert the coordinates to the custom LAEA projection
    polygon_laea = transform(transform_coordinates, polygon)

    # Calculate the area in square meters
    area_sqm = polygon_laea.area

    # Convert the area to square feet (1 square meter = 10.764 square feet)
    area_sqft = area_sqm * 10.764

    return area_sqft



### Call Function and add to dataframe
df_cleaned['foul_area_sqft'] = df_cleaned['foul'].apply(calculate_area)
df_cleaned['fop_area_sqft'] = df_cleaned['fop'].apply(calculate_area)

## Calculate the total area of the field and the ratio of foul area to field area
df_cleaned['field_area_sqft'] = df_cleaned['foul_area_sqft'] + df_cleaned['fop_area_sqft']
## Percentage foul area
df_cleaned['foul_area_per'] = df_cleaned['foul_area_sqft'] / df_cleaned['field_area_sqft']
## Fair to Foul Ratio
df_cleaned['fair_to_foul'] = df_cleaned['fop_area_sqft'] / df_cleaned['foul_area_sqft']


In [8]:
############# FENCE DISTANCE CALCULATION #############

from geopy.distance import great_circle
import numpy as np

def interpolate_points(start, end, length_ratio):
    start_np = np.array(start)
    end_np = np.array(end)
    return tuple(start_np + (end_np - start_np) * length_ratio)

def calculate_distances(home_plate, outfield_coords, num_points=30):
    def is_same_point(point1, point2, tolerance=1e-6):
        return abs(point1[0] - point2[0]) < tolerance and abs(point1[1] - point2[1]) < tolerance

    home_plate_lat_lon = (home_plate[1], home_plate[0])
    distances = []

    # Calculate total line length
    total_length = 0
    segments = []
    for i in range(len(outfield_coords) - 1):
        start = outfield_coords[i]
        end = outfield_coords[i + 1]
        if not is_same_point(home_plate, start) and not is_same_point(home_plate, end):
            segment_length = great_circle((start[1], start[0]), (end[1], end[0])).feet
            segments.append((start, end, segment_length))
            total_length += segment_length

    # Calculate the distance between equally spaced points
    spacing = total_length / (num_points - 1)

    # Interpolate points and calculate distances
    current_length = 0
    segment_index = 0
    for i in range(num_points):
        while segment_index < len(segments) - 1 and current_length > segments[segment_index][2]:
            current_length -= segments[segment_index][2]
            segment_index += 1

        start, end, segment_length = segments[segment_index]
        length_ratio = current_length / segment_length
        point = interpolate_points(start, end, length_ratio)
        distance = round(great_circle(home_plate_lat_lon, (point[1], point[0])).feet)
        distances.append(distance)

        current_length += spacing

    return distances

# Calculate distances for each row
df_cleaned['distances'] = df_cleaned.apply(lambda row: calculate_distances(row['home_plate'], row['fop']), axis=1)

# Calculate max, min, and average distances for each row
df_cleaned['max_distance'] = df_cleaned['distances'].apply(max)
df_cleaned['min_distance'] = df_cleaned['distances'].apply(min)
df_cleaned['avg_distance'] = df_cleaned['distances'].apply(lambda distances: sum(distances) / len(distances))
# get the median distance
df_cleaned['median_distance'] = df_cleaned['distances'].apply(lambda distances: np.median(distances))


In [9]:
######## CHECK BLOCK ########

## Check how long the distance list is for each row
df_cleaned['num_distances'] = df_cleaned['distances'].apply(len)

## Print the value counts for the 'num_distances' column
df_cleaned['num_distances'].value_counts()

30    127
Name: num_distances, dtype: int64

In [10]:
########## NOT NECESSARY FOR THIS PROJECT ##########

# ### Get Geolocation of each field based on home plate coordinates and return state and country
# ### This block takes a long time to run - will need to revisit
# ## up to ten minutes

# from geopy.geocoders import Nominatim
# from geopy.exc import GeocoderTimedOut, GeocoderServiceError
# from tqdm import tqdm

# geolocator = Nominatim(user_agent="baseball_field_locator")

# # Function to get location information
# def get_location_info(lng, lat):
#     try:
#         location = geolocator.reverse((lat, lng), timeout=10)
#         state = location.raw['address'].get('state', None)
#         country = location.raw['address'].get('country', None)
#         return state, country
#     except GeocoderTimedOut:
#         print(f"GeocoderTimedOut error for coordinates: ({lng}, {lat})")
#         return None, None
#     except GeocoderServiceError:
#         print(f"GeocoderServiceError for coordinates: ({lng}, {lat})")
#         return None, None

# # Extract the first coordinate for each field
# df_cleaned['lng'], df_cleaned['lat'] = zip(*df_cleaned['home_plate'].apply(lambda x: x))

# # Wrap the DataFrame apply function with tqdm for progress indication
# tqdm.pandas(desc="Processing coordinates")

# # Get state and country information for each field
# df_cleaned[['state', 'country']] = df_cleaned.progress_apply(lambda row: get_location_info(row['lng'], row['lat']), axis=1, result_type='expand')


In [11]:
df_cleaned.head()

,field,foul,fop,notes,home_plate,foul_area_sqft,fop_area_sqft,field_area_sqft,foul_area_per,fair_to_foul,distances,max_distance,min_distance,avg_distance,median_distance,num_distances
0,Muskegon Western Michigan Christian HS,"[(-86.2367956, 43.1609022), (-86.236754, 43.16...","[(-86.2367956, 43.1609022), (-86.2356235, 43.1...",None,"(-86.2367956, 43.1609022)",28662.155920,91282.061102,119944.217022,0.238962,3.184759,"[312, 314, 316, 320, 325, 331, 337, 345, 354, ...",382,301,343.066667,340.5,30
1,St Joseph HS,"[(-86.4923405, 42.0912197), (-86.4923378, 42.0...","[(-86.4923405, 42.0912197), (-86.4911402, 42.0...",no fence in foul outfield,"(-86.4923405, 42.0912197)",40944.376484,95222.837921,136167.214406,0.300692,2.325663,"[325, 330, 334, 338, 342, 345, 347, 350, 351, ...",361,325,347.766667,350.5,30
2,Petoskey High School,"[(-84.9386874, 45.36723030000001), (-84.938694...","[(-84.9386874, 45.36723030000001), (-84.937461...",None,"(-84.9386874, 45.36723030000001)",30317.378481,91940.911472,122258.289954,0.247978,3.032614,"[314, 315, 316, 319, 323, 327, 333, 341, 348, ...",365,314,340.166667,341.5,30
3,U of M - college,"[(-83.7420742, 42.2678568), (-83.7420447, 42.2...","[(-83.7420742, 42.2678568), (-83.7432557, 42.2...",None,"(-83.7420742, 42.2678568)",19408.615773,100786.703042,120195.318816,0.161476,5.192885,"[319, 323, 327, 333, 340, 348, 358, 368, 372, ...",394,309,357.000000,365.0,30
4,Sagniaw Valley State - college,"[(-83.9533999, 43.5122297), (-83.9533737, 43.5...","[(-83.9533999, 43.5122297), (-83.9521446, 43.5...",None,"(-83.9533999, 43.5122297)",39784.867681,106233.673037,146018.540718,0.272464,2.670203,"[332, 339, 345, 351, 357, 362, 367, 372, 375, ...",386,332,367.000000,372.5,30


In [12]:
######## CHECK AND WHATNOT BLOCK



In [13]:
## Function to create ranks for each column

def rank_fields(df):
    # Calculate the rank for each category
    df['max_distance_rank'] = df['max_distance'].rank(ascending=False, method='min')
    df['min_distance_rank'] = df['min_distance'].rank(ascending=False, method='min')
    df['avg_distance_rank'] = df['avg_distance'].rank(ascending=False, method='min')
    df['median_distance_rank'] = df['median_distance'].rank(ascending=False, method='min')
    df['field_area_rank'] = df['field_area_sqft'].rank(ascending=False, method='min')
    df['foul_area_rank'] = df['foul_area_sqft'].rank(ascending=False, method='min')
    df['fop_area_per_rank'] = df['fop_area_sqft'].rank(ascending=False, method='min')
    df['ratio_rank'] = df['fair_to_foul'].rank(ascending=False, method='min')

    return df



In [14]:
## Run Function

df_cleaned = rank_fields(df_cleaned)

In [15]:
#### Orienting the map to the home plate location ####

### Find the center of the field
def calculate_centroid(coords):
    x_coords = [coord[0] for coord in coords]
    y_coords = [coord[1] for coord in coords]
    centroid_x = sum(x_coords) / len(coords)
    centroid_y = sum(y_coords) / len(coords)
    return (centroid_x, centroid_y)


## Find the bearing between the home plate and the center of the field
import math

def calculate_bearing(point1, point2):
    lat1, lon1 = math.radians(point1[1]), math.radians(point1[0])
    lat2, lon2 = math.radians(point2[1]), math.radians(point2[0])

    d_lon = lon2 - lon1

    x = math.cos(lat2) * math.sin(d_lon)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(d_lon)

    bearing = math.degrees(math.atan2(x, y))
    bearing = (bearing + 360) % 360  # Normalize the bearing to the range [0, 360)

    return bearing

### Function to classify direction in laymans terms North, South, East, West, ect
def degrees_to_cardinal_direction(degrees):
    directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N']
    index = round(degrees / 45)
    return directions[index]


In [16]:
# Calculate the centroid of the outfield fence coordinates for each row
df_cleaned['fop_centroid'] = df_cleaned['fop'].apply(lambda coords: calculate_centroid(coords[1:]))

# Calculate the bearing between home plate and the centroid for each row
df_cleaned['field_orientation'] = df_cleaned.apply(lambda row: calculate_bearing(row['home_plate'], row['fop_centroid']), axis=1)

# Convert the bearing to a cardinal direction
df_cleaned['field_cardinal_direction'] = df_cleaned['field_orientation'].apply(degrees_to_cardinal_direction)



### All the geo transformation should take place above this

## starting the process of matching in data from other sources

In [17]:
######### RENAME field to park_name #########
###### SHould move up the file ######

df_cleaned.rename(columns={'field': 'park_name'}, inplace=True)

In [18]:
######### NEED TO RENAME TO WORK WITH NEXT BLOCK #########

hs_df = df_cleaned

# WORK DOWN FROM HERE

### FILL IN THE REST OF JSON WITH THE DATA FOR THE 2023 TOURNEY

In [19]:
####### OLD CODE OLD DATA #########

###### Matching block ######

## works with original 2022 table


from fuzzywuzzy import fuzz, process

def find_best_match(school_name, choices, score_cutoff=80):
    best_match = process.extractOne(school_name, choices, scorer=fuzz.token_sort_ratio, score_cutoff=score_cutoff)
    if best_match:
        return best_match[0]
    else:
        return None

# Read CSV files
### 2022 Enrollment and Division Date from MHSAA
######### Will want to find better data source
mhsaa_df = pd.read_csv('data/school_info/mhsaa_enrolment_2022.csv')

##### Nickname and Color Data from MHSAA that should be usefull
name_color_df = pd.read_csv('data\school_info\mhsaa_school_nickname_color_2020.csv')

# Get the list of park names from hs_df
park_names = hs_df['park_name'].tolist()

# Apply find_best_match function to create a new column 'best_match' in mhsaa_df
mhsaa_df['best_match'] = mhsaa_df['school_name'].apply(find_best_match, choices=park_names, score_cutoff=90)

## Pull the school_id, school_name, students, and division columns from mhsaa_df and add to hs_df
columns_to_extract = ['school_id', 'school_name', 'students', 'division']
for col in columns_to_extract:
    hs_df[col] = hs_df['park_name'].apply(lambda x: mhsaa_df.loc[mhsaa_df['best_match'] == x, col].iloc[0] if not mhsaa_df.loc[mhsaa_df['best_match'] == x, col].empty else None)

# Apply find_best_match function to create a new column 'best_match' in name_color_df
name_color_df['best_match'] = name_color_df['School'].apply(find_best_match, choices=park_names, score_cutoff=80)

## Pull the data from the name_color_df and add to hs_df (Nickname,Color1,Color2,Color3,Color4)
columns_to_extract = ['Nickname', 'Color1', 'Color2', 'Color3', 'Color4']
for col in columns_to_extract:
    hs_df[col] = hs_df['park_name'].apply(lambda x: name_color_df.loc[name_color_df['best_match'] == x, col].iloc[0] if not name_color_df.loc[name_color_df['best_match'] == x, col].empty else None)

# Drop the 'best_match' columns
# hs_df.drop(columns=['best_match'], inplace=True)

## Take a look at the new hs_df
hs_df.head()


,park_name,foul,fop,notes,home_plate,foul_area_sqft,fop_area_sqft,field_area_sqft,foul_area_per,fair_to_foul,...,field_cardinal_direction,school_id,school_name,students,division,Nickname,Color1,Color2,Color3,Color4
0,Muskegon Western Michigan Christian HS,"[(-86.2367956, 43.1609022), (-86.236754, 43.16...","[(-86.2367956, 43.1609022), (-86.2356235, 43.1...",None,"(-86.2367956, 43.1609022)",28662.155920,91282.061102,119944.217022,0.238962,3.184759,...,NE,3490.0,Muskegon - Western Michigan Christian HS,243.0,C,Warriors,Forest Green,White,NaN,NaN
1,St Joseph HS,"[(-86.4923405, 42.0912197), (-86.4923378, 42.0...","[(-86.4923405, 42.0912197), (-86.4911402, 42.0...",no fence in foul outfield,"(-86.4923405, 42.0912197)",40944.376484,95222.837921,136167.214406,0.300692,2.325663,...,NE,2708.0,St Joseph HS,1020.0,A,Bears,Maize,Blue,NaN,NaN
2,Petoskey High School,"[(-84.9386874, 45.36723030000001), (-84.938694...","[(-84.9386874, 45.36723030000001), (-84.937461...",None,"(-84.9386874, 45.36723030000001)",30317.378481,91940.911472,122258.289954,0.247978,3.032614,...,NE,NaN,None,NaN,None,None,None,None,None,NaN
3,U of M - college,"[(-83.7420742, 42.2678568), (-83.7420447, 42.2...","[(-83.7420742, 42.2678568), (-83.7432557, 42.2...",None,"(-83.7420742, 42.2678568)",19408.615773,100786.703042,120195.318816,0.161476,5.192885,...,SW,NaN,None,NaN,None,None,None,None,None,NaN
4,Sagniaw Valley State - college,"[(-83.9533999, 43.5122297), (-83.9533737, 43.5...","[(-83.9533999, 43.5122297), (-83.9521446, 43.5...",None,"(-83.9533999, 43.5122297)",39784.867681,106233.673037,146018.540718,0.272464,2.670203,...,NE,NaN,None,NaN,None,None,None,None,None,NaN


In [20]:
df = df_cleaned

df.head()

,park_name,foul,fop,notes,home_plate,foul_area_sqft,fop_area_sqft,field_area_sqft,foul_area_per,fair_to_foul,...,field_cardinal_direction,school_id,school_name,students,division,Nickname,Color1,Color2,Color3,Color4
0,Muskegon Western Michigan Christian HS,"[(-86.2367956, 43.1609022), (-86.236754, 43.16...","[(-86.2367956, 43.1609022), (-86.2356235, 43.1...",None,"(-86.2367956, 43.1609022)",28662.155920,91282.061102,119944.217022,0.238962,3.184759,...,NE,3490.0,Muskegon - Western Michigan Christian HS,243.0,C,Warriors,Forest Green,White,NaN,NaN
1,St Joseph HS,"[(-86.4923405, 42.0912197), (-86.4923378, 42.0...","[(-86.4923405, 42.0912197), (-86.4911402, 42.0...",no fence in foul outfield,"(-86.4923405, 42.0912197)",40944.376484,95222.837921,136167.214406,0.300692,2.325663,...,NE,2708.0,St Joseph HS,1020.0,A,Bears,Maize,Blue,NaN,NaN
2,Petoskey High School,"[(-84.9386874, 45.36723030000001), (-84.938694...","[(-84.9386874, 45.36723030000001), (-84.937461...",None,"(-84.9386874, 45.36723030000001)",30317.378481,91940.911472,122258.289954,0.247978,3.032614,...,NE,NaN,None,NaN,None,None,None,None,None,NaN
3,U of M - college,"[(-83.7420742, 42.2678568), (-83.7420447, 42.2...","[(-83.7420742, 42.2678568), (-83.7432557, 42.2...",None,"(-83.7420742, 42.2678568)",19408.615773,100786.703042,120195.318816,0.161476,5.192885,...,SW,NaN,None,NaN,None,None,None,None,None,NaN
4,Sagniaw Valley State - college,"[(-83.9533999, 43.5122297), (-83.9533737, 43.5...","[(-83.9533999, 43.5122297), (-83.9521446, 43.5...",None,"(-83.9533999, 43.5122297)",39784.867681,106233.673037,146018.540718,0.272464,2.670203,...,NE,NaN,None,NaN,None,None,None,None,None,NaN
